# Ex06 - Introdução ao OpenGL

Nesta atividade, vocês vão exercitar os primeiros conceitos de OpenGL. Para evitar erros de execução, utilize apenas uma célula de código para cada parte desta atividade.

### Parte 1 - Renderização de objetos simples

Crie um programa que reproduza uma imagem semelhante à imagem abaixo (triângulos nos cantos e um quadrado no centro). O tamanho e cor dos objetos e a cor do background é livre, mas as arestas dos triângulos devem estar coladas nos cantos da janela. Você pode usar como base os notebooks notebook ([17_Primeiro_programa_OpenGL](17_Primeiro_programa_OpenGL.ipynb)), ([18_Configurando_uma_cor_unica_para_todos_os_vertices](18_Configurando_uma_cor_unica_para_todos_os_vertices.ipynb)) e ([19_Renderizando_mais_de_um_objeto](19_Renderizando_mais_de_um_objeto.ipynb)).

![title](cg/images/ex6_image.png)

In [3]:
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v1 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v1 import ModelRenderer

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        # posição dos vértices do quadrado
        square_vertex_position = np.array([
            -0.5, -0.5, 0.0, 1.0, # Triângulo 1
             0.5, -0.5, 0.0, 1.0,
            -0.5,  0.5, 0.0, 1.0,
             0.5, -0.5, 0.0, 1.0, # Triângulo 2
             0.5,  0.5, 0.0, 1.0,
            -0.5,  0.5, 0.0, 1.0],
            dtype=np.float32)
        
        # posição dos vértices do triângulo inferior esquerdo
        bottom_left_triangle_vertex_position = np.array([
            -1.0, -0.5, 0.0, 1.0, 
            -1.0, -1.0, 0.0, 1.0,
            -0.5, -1.0, 0.0, 1.0],
            dtype=np.float32)
        
        # posição dos vértices do triângulo superior direito
        upper_right_triangle_vertex_position = np.array([
             1.0,  0.5, 0.0, 1.0, 
             1.0,  1.0, 0.0, 1.0,
             0.5,  1.0, 0.0, 1.0],
            dtype=np.float32)
        
        # cria os objetos responsáveis por carregar os dados para a GPU e renderizá-los
        self.squareRenderer = ModelRenderer(square_vertex_position)
        self.bottomLeftRenderer = ModelRenderer(bottom_left_triangle_vertex_position)
        self.upperRightRenderer = ModelRenderer(upper_right_triangle_vertex_position)
        
        # cria um shader program simples
        self.shaderProgram = SimpleShaderProgram()
        
        # ativa o shader programa para configurar uma cor única para todos os vértices
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.release()

        # recupera o endereços da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        
        # configura os dados dos modelos para serem os dados de entrada do shader program
        self.squareRenderer.setVertexPositionLoc(position_loc)
        self.bottomLeftRenderer.setVertexPositionLoc(position_loc)
        self.upperRightRenderer.setVertexPositionLoc(position_loc)

    def paintGL(self):
        
        # configura a cor de background
        gl.glClearColor(0, 0, 0, 1)
        
        # limpa o background com a cor especificada
        gl.glClear(gl.GL_COLOR_BUFFER_BIT)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # mudar a cor no shader e renderiza o quadrado no centro
        self.shaderProgram.setUniformColor(np.array([0.0, 0.0, 0.5, 1.0], dtype=np.float32))
        self.squareRenderer.render()
        
        # mudar a cor no shader e renderiza o triângulo no canto inferior esquerdo
        self.shaderProgram.setUniformColor(np.array([0.0, 0.5, 0.0, 1.0], dtype=np.float32))
        self.bottomLeftRenderer.render()
        
        # mudar a cor no shader e renderiza o triângulo no canto superior direito
        self.shaderProgram.setUniformColor(np.array([0.5, 0.0, 0.0, 1.0], dtype=np.float32))
        self.upperRightRenderer.render()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()

    def resizeGL(self, width, height):
    
        gl.glViewport(0, 0, width, height)

def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(640, 480)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Parte 2 - Animação

Crie um programa que tenha um objeto (triângulo, quadrado, etc) desenhado no centro, e que as cores deste objeto e do background mudam ao longo do tempo. Você pode usar como base o notebook ([20_Animando_as_cores_dos_triangulos](20_Animando_as_cores_dos_triangulos.ipynb))

In [5]:
import time
import math
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v1 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v1 import ModelRenderer

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        # posição dos vértices do quadrado
        square_vertex_position = np.array([
            -0.5, -0.5, 0.0, 1.0, # Triângulo 1
             0.5, -0.5, 0.0, 1.0,
            -0.5,  0.5, 0.0, 1.0,
             0.5, -0.5, 0.0, 1.0, # Triângulo 2
             0.5,  0.5, 0.0, 1.0,
            -0.5,  0.5, 0.0, 1.0],
            dtype=np.float32)
        
        # cria um objeto responsável por carregar os dados para a GPU e renderizá-los
        self.squareRenderer = ModelRenderer(square_vertex_position)
        
        # cria um shader program simples
        self.shaderProgram = SimpleShaderProgram()
        
        # ativa o shader programa para configurar uma cor única para todos os vértices
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.release()

        # recupera o endereços da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        
        # configura os dados do modelo para serem os dados de entrada do shader program
        self.squareRenderer.setVertexPositionLoc(position_loc)

        # armazena o momento que o programa começou
        self.startTime = time.time()
        
    def paintGL(self):
        
        # calcula o tempo de execução do programa
        self.currentTime = time.time()
        time_difference = self.currentTime - self.startTime
        
        # calcula o fator de interpolação
        interpolation_factor = (math.sin(time_difference) + 1) / 2
        
        color_01 = np.array([0.0, 0.0, 1.0, 1.0], dtype=np.float32)
        color_02 = np.array([1.0, 0.0, 0.0, 1.0], dtype=np.float32)
        
        #calcula as novas cores
        mixed_color_01 = interpolation_factor * color_01 + (1 - interpolation_factor) * color_02
        mixed_color_02 = interpolation_factor * color_02 + (1 - interpolation_factor) * color_01
        
        # configura a nova cor de background
        gl.glClearColor(mixed_color_01[0], mixed_color_01[1], mixed_color_01[2], mixed_color_01[3])
        
        # limpa o background com a cor especificada
        gl.glClear(gl.GL_COLOR_BUFFER_BIT)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # mudar a cor no shader e renderiza o quadrado
        self.shaderProgram.setUniformColor(mixed_color_02)
        self.squareRenderer.render()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()

    def resizeGL(self, width, height):
    
        gl.glViewport(0, 0, width, height)

def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(640, 480)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Parte 3 - Evento de teclado

Crie um programa que tenha um objeto (triângulo, quadrado, etc) desenhado e utilize a setas do teclado para deslocar este objeto na janela. O objeto deve se deslocar na direção da teclada pressionada. Você pode usar como base o notebook ([24_Eventos_do_teclado.ipynb](24_Eventos_do_teclado.ipynb))

In [4]:
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL, QtCore
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v1 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v1 import ModelRenderer

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        # posição de cada vértice dos triângulos
        self.squareVertexPosition = np.array([
            -0.5, -0.5, 0.0, 1.0, # Triângulo 1
             0.5, -0.5, 0.0, 1.0,
            -0.5,  0.5, 0.0, 1.0,
             0.5, -0.5, 0.0, 1.0, # Triângulo 2
             0.5,  0.5, 0.0, 1.0,
            -0.5,  0.5, 0.0, 1.0],
            dtype=np.float32)
        
        # cria um objeto responsável por carregar os dados para a GPU e renderizá-los
        self.squareRenderer = ModelRenderer(self.squareVertexPosition)
        
        # cria um shader program simples
        self.shaderProgram = SimpleShaderProgram()
        
        # ativa o shader programa para configurar uma cor única para todos os vértices
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.setUniformColor(np.array([0.0, 0.0, 0.5, 1.0], dtype=np.float32))
        self.shaderProgram.release()

        # recupera o endereços da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        
        # configura os dados do modelo para serem os dados de entrada do shader program
        self.squareRenderer.setVertexPositionLoc(position_loc)

    def paintGL(self):
        
        # configura a cor de background
        gl.glClearColor(0, 0, 0, 1)
        
        # limpa o background com a cor especificada
        gl.glClear(gl.GL_COLOR_BUFFER_BIT)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        self.squareRenderer.render()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()

    def resizeGL(self, width, height):
    
        gl.glViewport(0, 0, width, height)
    
    def keyPressEvent(self, event):
        super(MyWidget, self).keyPressEvent(event)
        
        # passo 
        step = 0.1
        
        # verifica se foi precionada a tecla de seta 'para cima'
        if event.key() == QtCore.Qt.Key_Up:
            if(np.all(self.squareVertexPosition[1::4] < 1.0)):
                self.squareVertexPosition[1::4] += step

        # verifica se foi precionada a tecla de seta 'para baixo'
        elif event.key() == QtCore.Qt.Key_Down:
            if(np.all(self.squareVertexPosition[1::4] > -1.0)):
                self.squareVertexPosition[1::4] -= step
        
        # verifica se foi precionada a tecla de seta 'para esquerda'
        elif event.key() == QtCore.Qt.Key_Left:
            if(np.all(self.squareVertexPosition[0::4] > -1.0)):
                self.squareVertexPosition[0::4] -= step
        
        # verifica se foi precionada a tecla de seta 'para direita'
        elif event.key() == QtCore.Qt.Key_Right:
            if(np.all(self.squareVertexPosition[0::4] < 1.0)):
                self.squareVertexPosition[0::4] += step
        
        # atualiza os novos dados de posição na GPU
        self.squareRenderer.updateVertexPositions(self.squareVertexPosition)

def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(640, 480)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
